# Use Matminer to download dataset and calculate intrinsic hardness

## Import libraries

In [56]:
import sys
from pathlib import Path

from tqdm.notebook import tqdm
from matminer.datasets import load_dataset
from pymatgen.io.cif import CifWriter

sys.path.append('..')
import bond_detectors, hardness_calculators

In [ ]:
data_dir = Path.cwd().parent / "data"

## Import matminer brgoch dataset

In [53]:
df = load_dataset("brgoch_superhard_training")
df.dropna(subset=["structure", "material_id"], inplace=True)
df

,formula,bulk_modulus,shear_modulus,composition,material_id,structure,brgoch_feats,suspect_value
0,AlPt3,225.230461,91.197748,"(Al, Pt)",mp-188,"[[0. 0. 0.] Al, [0. 1.96140395 1.96140...","{'atomic_number_feat_1': 123.5, 'atomic_number...",False
1,Mn2Nb,232.696340,74.590157,"(Mn, Nb)",mp-12659,[[-2.23765223e-08 1.42974191e+00 5.92614104e...,"{'atomic_number_feat_1': 45.5, 'atomic_number_...",False
2,HfO2,204.573433,98.564374,"(Hf, O)",mp-352,"[[2.24450185 3.85793022 4.83390736] O, [2.7788...","{'atomic_number_feat_1': 44.0, 'atomic_number_...",False
3,Cu3Pt,159.312640,51.778816,"(Cu, Pt)",mp-12086,"[[0. 1.86144248 1.86144248] Cu, [1.861...","{'atomic_number_feat_1': 82.5, 'atomic_number_...",False
4,Mg3Pt,69.637565,27.588765,"(Mg, Pt)",mp-18707,"[[0. 0. 2.73626461] Mg, [0. ...","{'atomic_number_feat_1': 57.0, 'atomic_number_...",False
...,...,...,...,...,...,...,...,...
2569,MgAsPt5,181.857661,70.362793,"(Mg, As, Pt)",mp-865146,"[[1.99379423 1.99379423 0. ] Mg, [0. ...","{'atomic_number_feat_1': 145.0, 'atomic_number...",False
2570,YCBr,22.970769,9.733656,"(Y, C, Br)",mp-643367,"[[0.66584097 1.92338676 8.4823332 ] Y, [-0.645...","{'atomic_number_feat_1': 26.6666666666667, 'at...",False
2571,NaTiO2,98.358957,70.565034,"(Na, Ti, O)",mp-7002,"[[ 3.31024633 2.00832854 11.99124842] O, [1.0...","{'atomic_number_feat_1': 16.3333333333333, 'at...",False
2572,KCuSe,34.111565,21.586576,"(K, Cu, Se)",mp-7435,"[[0. 0. 0.] K, [0. 0. 4.903557...","{'atomic_number_feat_1': 27.3333333333333, 'at...",False


## Calculate hardness

### Pugh's hardness
Chen, X.-Q., Niu, H., Li, D. & Li, Y., 2011. Modeling hardness of polycrystalline materials and bulk metallic glasses. Intermetallics 19, 1275–1281.

$
\displaystyle H_V=2\left( \frac{G^3}{K^2} \right)^{0.585} - 3
$

In [54]:
df["pugh_hardness"] = 2*((df["shear_modulus"]**3)/(df["bulk_modulus"]**2))**0.585 - 3

### CAS hardness
Cheenady, A.A., Awasthi, A., Subhash, G., 2021. Intrinsic hardness of covalent crystals: a unified multiparametric framework. J. Mater. Sci. 56, 11711–11722.

$
\displaystyle H_{C} = 986 \left(\frac{N_b}{V}\right)^{0.844} \left[ \prod_{ab=1}^N Z_{ij}^{0.006} d^{-3.18}  e^{-2.44fi_{ab}} \right]^{1/N} \\
\displaystyle Z_{ab} = \frac{\chi_a}{n_a} \frac{\chi_b}{n_b} \\
\displaystyle fi_{ab} = \left( \frac{\chi_a - \chi_b}{\chi_a + \chi_b} \right)^2
$

In [ ]:
for row in tqdm(df.itertuples(), total=len(df)):
    structure = row.structure
    bonds = bond_detectors.detect_bonds(structure)
    hardness = hardness_calculators.calculate_hardness(structure, bonds, model="CAS")
df["bonds"] = bonds
df["hardness"] = hardness

  0%|          | 0/2540 [00:00<?, ?it/s]

/home/salil.bavdekar/.conda/envs/mp_pymatgen/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:4133: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
/home/salil.bavdekar/.conda/envs/mp_pymatgen/lib/python3.10/site-packages/pymatgen/analysis/local_env.py:3936: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


### Save DataFrame

In [ ]:
df.to_csv(data_dir / "data.csv")
df.to_pickle(data_dir / "data.pkl")
df.to_excel(data_dir / "data.xlsx")
df

## Save as CIF file

In [47]:
for row in tqdm(df.itertuples(), total=len(df)):
    structure = row.structure
    savepath = data_dir / f"{row.material_id}.cif"
    try:
        CifWriter(struct=structure).write_file(savepath)
    except AttributeError:
        print(f"AttributeError: {row.formula}")

  0%|          | 0/2574 [00:00<?, ?it/s]

AttributeError: MnO2
AttributeError: MnS
AttributeError: NiAs
AttributeError: TiNi
AttributeError: WO2
AttributeError: TiNCl
AttributeError: YMgPd
AttributeError: K2PdCl4
AttributeError: MgPPt5
AttributeError: CaMnO3
AttributeError: CoAgO2
AttributeError: MgNiSb
AttributeError: CaNiN
AttributeError: Ca2ZnN2
AttributeError: YPPt
AttributeError: AlFeF5
AttributeError: Ba(MgSb)2
AttributeError: Mg3NF3
AttributeError: TiCuSi
AttributeError: Zr2SbP
AttributeError: TaNiGe
AttributeError: Ca(ZnSb)2
AttributeError: Zr2SN2
AttributeError: ZrSnTe
AttributeError: TiAlCo2
AttributeError: CaFeO2
AttributeError: ZrCoSn
AttributeError: Ca3AlSb3
AttributeError: SrAl9Co2
AttributeError: Ca(GePd)2
AttributeError: BaZnGe
AttributeError: Cs2HfF6
AttributeError: Mg2NF
AttributeError: YNiSn
